In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import bqplot as bq
import ipywidgets as ipy
import calendar
import squarify
import matplotlib
import bqplot.market_map
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
df_adrd = pd.read_csv("Accidental_Drug_Related_Deaths_2012-2018.csv")

In [3]:
df_adrd['Year'] = pd.to_datetime(df_adrd['Date']).dt.year
df_adrd['Year'].fillna(0, inplace=True)
df_adrd['Year'] = df_adrd['Year'].apply(int)

In [4]:
df_top_drugs = pd.DataFrame()

df_top_drugs['Heroin'] = df_adrd.groupby('Year').Heroin.count()

df_top_drugs['Cocaine'] = df_adrd.groupby('Year').Cocaine.count()

df_top_drugs['Fentanyl'] = df_adrd.groupby('Year').Fentanyl.count()

df_top_drugs['Benzodiazepine'] = df_adrd.groupby('Year').Benzodiazepine.count()

df_top_drugs['Ethanol'] = df_adrd.groupby('Year').Ethanol.count()

In [5]:
df_top_drugs.reset_index(inplace = True)
df_top_drugs = df_top_drugs[df_top_drugs.Year != 0]

### Selectors

#### 1. Drugs dropdown

In [6]:
drop_drug = ipy.Dropdown(
            options=['Benzodiazepine', 'Cocaine', 'Ethanol', 'Fentanyl', 'Heroin','FentanylAnalogue','Oxycodone','Oxymorphone',
                    'Hydrocodone','Methadone','Amphet','Tramad','Morphine_NotHeroin','Hydromorphone','OpiateNOS','AnyOpioid'],
            value='Benzodiazepine',
            disabled=False
        )

#### 2. City market map

In [7]:
top_cities = df_adrd.groupby("DeathCity").count()['ID'].nlargest(12)

color_scale = bq.ColorScale(scheme = "Blues")
color_axis = bq.ColorAxis(scale = color_scale, label = "Death Cities")

my_map = bq.market_map.MarketMap(
    names = top_cities.index.values,
    ref_data = top_cities,
    color = top_cities,
    scales = {'color': color_scale},
    axes = [color_axis],
#     rows=2,
#     cols=6,
#     min_aspect_ratio=2
    max_aspect_ratio=1.33

)

In [8]:
df_adrd.dropna(subset=['Date'], inplace=True)
df_adrd['Date'] = pd.to_datetime(df_adrd['Date'], format='%m/%d/%Y %I:%M:%S %p')
df_adrd['Day of the week'] = df_adrd['Date'].dt.weekday
df_adrd.dropna(subset=['Day of the week'], inplace = True)
df_adrd['Day of the week'] = df_adrd['Day of the week'].astype(int)
df_adrd['Day of the week'] = df_adrd['Day of the week'].apply(lambda x: calendar.day_abbr[x])

In [9]:
initial_df = df_adrd[df_adrd[drop_drug.value] == 'Y']
drug_df = df_adrd[df_adrd['Benzodiazepine'] == 'Y']
new_df_weekday = pd.DataFrame()
new_df_weekday = drug_df.groupby(['Day of the week']).ID.count().to_frame().reindex(['Sun',
                                                                                 'Mon',
                                                                                 'Tue',
                                                                                 'Wed',
                                                                                 'Thu',
                                                                                 'Fri',
                                                                                 'Sat'])

In [10]:
df_adrd['Year'] = pd.to_datetime(df_adrd['Date']).dt.year
df_adrd['Year'].fillna(0, inplace=True)
df_adrd['Year'] = df_adrd['Year'].apply(int)

In [11]:
df_drugs = pd.DataFrame()
df_drugs['Heroin'] = df_adrd.groupby('Year').Heroin.count()
df_drugs['Cocaine'] = df_adrd.groupby(['Year']).Cocaine.count()
df_drugs['Fentanyl'] = df_adrd.groupby(['Year']).Fentanyl.count()
df_drugs['Benzodiazepine'] = df_adrd.groupby(['Year']).Benzodiazepine.count()
df_drugs['Ethanol'] = df_adrd.groupby(['Year']).Ethanol.count()
df_drugs['FentanylAnalogue'] = df_adrd.groupby(['Year']).FentanylAnalogue.count()
df_drugs['Oxycodone'] = df_adrd.groupby(['Year']).Oxycodone.count()
df_drugs['Oxymorphone'] = df_adrd.groupby(['Year']).Oxymorphone.count()
df_drugs['Hydrocodone'] = df_adrd.groupby(['Year']).Hydrocodone.count()
df_drugs['Methadone'] = df_adrd.groupby(['Year']).Methadone.count()
df_drugs['Amphet'] = df_adrd.groupby(['Year']).Amphet.count()
df_drugs['Tramad'] = df_adrd.groupby(['Year']).Tramad.count()
df_drugs['Morphine_NotHeroin'] = df_adrd.groupby(['Year']).Morphine_NotHeroin.count()
df_drugs['Hydromorphone'] = df_adrd.groupby(['Year']).Hydromorphone.count()
df_drugs['OpiateNOS'] = df_adrd.groupby(['Year']).OpiateNOS.count()
df_drugs['AnyOpioid'] = df_adrd.groupby(['Year']).AnyOpioid.count()

In [12]:
df_drugs.reset_index(inplace=True)

# DROPDOWN

In [13]:
drop_drug = ipy.Dropdown(
            options=['Benzodiazepine', 'Cocaine', 'Ethanol', 'Fentanyl', 'Heroin','FentanylAnalogue','Oxycodone','Oxymorphone',
                    'Hydrocodone','Methadone','Amphet','Tramad','Morphine_NotHeroin','Hydromorphone','OpiateNOS','AnyOpioid'],
            value='Benzodiazepine',
            disabled=False
        )

# LINE CHART 

In [14]:
x_ord_lc = bq.OrdinalScale()
y_lin_lc = bq.LinearScale()

x_ax_lc = bq.Axis(scale = x_ord_lc, label = 'Year')
y_ax_lc = bq.Axis(scale = y_lin_lc, orientation = 'vertical', label = 'Number of Deaths')

x_data_lc = list(df_drugs.Year)
y_data_lc = df_drugs[str(drop_drug.value)]

line = bq.Lines(x=x_data_lc,
                y=y_data_lc,
                scales={'x': x_ord_lc, 'y': y_lin_lc},
                stroke_width=3,
                colors=['teal'],
                display_legend=True,
                labels=[str(drop_drug.value)]
               )


year_fig = bq.Figure(marks = [line], axes = [x_ax_lc, y_ax_lc], legend_location='top-left',
               title = 'Death rate by drugs over years')
year_fig.layout.width = '600px'

# BAR CHART - WEEK

In [15]:
x_ord_bc = bq.OrdinalScale()
y_lin_bc = bq.LinearScale()

ax_x_bc = bq.Axis(scale=x_ord_bc, label = 'Days of a week', grid_lines = 'none')
ax_y_bc = bq.Axis(scale=y_lin_bc, label = 'Drug caused deaths', grid_lines = 'none', orientation='vertical')

bar = bq.Bars(x=list(new_df_weekday.index), y=list(new_df_weekday['ID']), scales={'x': x_ord_bc, 'y': y_lin_bc},
              colors = ['Orange'], padding = 0.2)

week_fig = bq.Figure(marks=[bar], axes=[ax_x_bc, ax_y_bc], title = 'Deaths/weekday from 2012 - 2018')
week_fig.layout.width = '600px'

# BAR CHART - LOCATION

In [16]:
df_loc = pd.DataFrame()
df_loc['Count'] = df_adrd.groupby('Location').count()['ID']

In [17]:
df_loc.reset_index(inplace=True)

In [18]:
x_ord_loc = bq.OrdinalScale()
y_lin_loc = bq.LinearScale()

x_data_loc = list(df_loc['Location'])
y_data_loc = df_loc['Count']

bar_loc = bq.Bars(x=x_data_loc, y=y_data_loc, scales={'x': x_ord_loc, 'y': y_lin_loc})

ax_x_loc = bq.Axis(scale=x_ord_loc, grid_lines='solid', label='X')
ax_y_loc = bq.Axis(scale=y_lin_loc, orientation='vertical', grid_lines='solid', label='Y')

fig_loc = bq.Figure(marks=[bar_loc], axes=[ax_x_loc, ax_y_loc], title = 'Title')
fig_loc.layout.width = '600px'

# ON CHANGE

In [19]:
def on_change_city(event):
    city_df = df_adrd[(df_adrd['DeathCity'].isin(event['new'])) & (df_adrd[drop_drug.value] == 'Y')]
    new_df_weekday = city_df.groupby(['Day of the week']).ID.count().to_frame().reindex(['Sun',
                                                                                         'Mon',
                                                                                         'Tue',
                                                                                         'Wed',
                                                                                         'Thu',
                                                                                         'Fri',
                                                                                         'Sat'])
    bar.y = list(new_df_weekday['ID'])
    line.y = df_adrd.query('DeathCity == '+str(event["new"])+' & '+drop_drug.value+' == "Y"').groupby(['Year'])['ID'].count()
    bar_loc.y = df_adrd.query('DeathCity == '+str(event["new"])+' & '+drop_drug.value+' == "Y"').groupby(['Location'])['ID'].count()




    
    
def on_change_drug(event):
    drug_df = df_adrd[(df_adrd[event['new']] == 'Y') & (df_adrd['DeathCity'].isin(my_map.selected))]
    new_df_weekday = pd.DataFrame()
    new_df_weekday = drug_df.groupby(['Day of the week']).ID.count().to_frame().reindex(['Sun',
                                                                                         'Mon',
                                                                                         'Tue',
                                                                                         'Wed',
                                                                                         'Thu',
                                                                                         'Fri',
                                                                                         'Sat'])
    bar.y = list(new_df_weekday['ID'])
    line.y = df_adrd.query('DeathCity == '+str(my_map.selected)+' & '+event["new"]+' == "Y"').groupby(['Year'])['ID'].count()
    line.labels = [drop_drug.value]
    bar_loc.y = df_adrd.query('DeathCity == '+str(my_map.selected)+' & '+event["new"]+' == "Y"').groupby(['Location'])['ID'].count()



my_map.observe(on_change_city, ['selected'])
drop_drug.observe(on_change_drug, ['value'])


In [20]:
# display(ipy.HBox([drop_drug]))
# display(ipy.HBox([my_map, fig_loc]))
# display(ipy.HBox([week_fig, year_fig]))
display(ipy.HBox([drop_drug]))
display(ipy.HBox([my_map, fig_loc]))
display(ipy.HBox([week_fig, year_fig]))

In [21]:
df_adrd['Sex'].fillna(value = 'Unknown', inplace=True)

In [22]:
# A dataframe containing gender wise grouped drug deaths
df_sex = pd.DataFrame()
df_sex['Heroin'] = df_adrd.groupby(['Sex']).Heroin.count()
df_sex['Cocaine'] = df_adrd.groupby(['Sex']).Cocaine.count()
df_sex['Fentanyl'] = df_adrd.groupby(['Sex']).Fentanyl.count()
df_sex['FentanylAnalogue'] = df_adrd.groupby(['Sex']).FentanylAnalogue.count()
df_sex['Oxycodone'] = df_adrd.groupby(['Sex']).Oxycodone.count()
df_sex['Oxymorphone'] = df_adrd.groupby(['Sex']).Oxymorphone.count()
df_sex['Ethanol'] = df_adrd.groupby(['Sex']).Ethanol.count()
df_sex['Hydrocodone'] = df_adrd.groupby(['Sex']).Hydrocodone.count()
df_sex['Benzodiazepine'] = df_adrd.groupby(['Sex']).Benzodiazepine.count()
df_sex['Methadone'] = df_adrd.groupby(['Sex']).Methadone.count()
df_sex['Amphet'] = df_adrd.groupby(['Sex']).Amphet.count()
df_sex['Tramad'] = df_adrd.groupby(['Sex']).Tramad.count()
df_sex['Morphine_NotHeroin'] = df_adrd.groupby(['Sex']).Morphine_NotHeroin.count()
df_sex['Hydromorphone'] = df_adrd.groupby(['Sex']).Hydromorphone.count()
df_sex['Other'] = df_adrd.groupby(['Sex']).Other.count()
df_sex['OpiateNOS'] = df_adrd.groupby(['Sex']).OpiateNOS.count()
df_sex['AnyOpioid'] = df_adrd.groupby(['Sex']).AnyOpioid.count()

In [23]:
tdf_sex = df_sex.transpose()

In [24]:
# Using bqplot to make a bar chart for easier comparison
sex_dropdown = ipy.Dropdown(options = ['Male', 'Female', 'Unknown'],
                            value = 'Male',
                            disabled = False,
                           description = 'Select Gender:')

x_sc = bq.OrdinalScale()
y_sc = bq.LinearScale()

x_ax = bq.Axis(scale = x_sc, offset = {'value':'100px'}, orientation = 'vertical')
y_ax = bq.Axis(scale = y_sc, grid_lines = 'dashed', label = 'Drug caused deaths')

x_data = list(tdf_sex.index)
y_data = np.array(tdf_sex[sex_dropdown.value])

bars = bq.marks.Bars(x = x_data,
              y = y_data,
              colors = ['red'],
              scales={'x':x_sc, 'y':y_sc},
              padding = 0.1,
               orientation = 'horizontal'
              )

fig = bq.Figure(marks = [bars], axes = [x_ax, y_ax], title = 'Drug effects on Males', padding_y = 0.1)
fig.layout.height = '700px'
fig.layout.width = '1000px'


def on_drop_change(event):
    bars.y = np.array(tdf_sex[event['new']])
    fig.title = 'Drug effects on ' + event['new'] + 's' 
    
sex_dropdown.observe(on_drop_change, ['value'])
    
display(ipy.VBox([sex_dropdown, fig]))

In [25]:
# import numpy as np
# from bqplot import *
# from IPython.display import Javascript, display, clear_output
# import ipywidgets as widgets

# size = 100
# scale = 100.
# np.random.seed(0)
# x_data = np.arange(size)
# y_data = np.cumsum(np.random.randn(size)  * scale)

# x_sc = LinearScale()
# y_sc = LinearScale()

# ax_x = Axis(label='X', scale=x_sc, grid_lines='solid')
# ax_y = Axis(label='Y', scale=y_sc, orientation='vertical', grid_lines='solid')

# line = Lines(x=x_data, y=x_data, scales={'x': x_sc, 'y': y_sc})
# figy=[]
# for i in range(2):
#     figx=[]
#     for j in range(3):
#         figx.append(bq.Figure(axes=[ax_x, ax_y], marks=[line], title='Example' + str(i*3+j), fig_margin = dict(top=30, bottom=10, left=20, right=20)))
#     figy.append(widgets.HBox(figx)) 
# display(widgets.VBox(figy, align_content = 'stretch'))